<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/Feed%20Forward%20NN/Doppler%20Effect/NN_CFO_w_AWGN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import scipy.io as spio
import math
import time
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
#Train model
%cd /content/drive/My Drive/NN\ Input/Doppler/AWGN/Train\ Data
start_time = time.time()

#parameters
num_classes = 2
batch_size = 64
epochs = 100

SNRs = [-2,10,23]
del_f = [5,20,40,70]

for SNR in SNRs:
    for ratio in del_f:

        matname = 'DVBS_dopp_ratio_%d_SNR_%d' % (ratio, SNR)
        print(matname)

        mat = spio.loadmat(matname, squeeze_me=True)
        x_train = mat['x_train']
        x_valid = mat['val_data']
        y_valid = mat['val_target']
        y_train = mat['y_train']

         #cast to float32
        x_train = x_train.astype('float32')
        x_valid = x_valid.astype('float32')
        #x_test = x_test.astype('float32') 

        print(x_train.shape, 'train samples')
        print(x_valid.shape, 'valid samples')

        print(y_train.shape, 'train labels')
        print(y_valid.shape, 'valid labels')

        print('Label Examples:\n', y_train)
        print('Input Examples:\n', x_train)

        # Define the network
        model = Sequential()
        model.add(Dense(50, activation='elu', input_dim=18))
        model.add(Dense(50, activation='elu'))
        model.add(Dense(num_classes, activation='tanh'))

        model.summary()

        model.compile(loss=keras.metrics.mean_squared_error,
                    optimizer=SGD(learning_rate=0.1),
                    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=2,
                            validation_data=(x_valid, y_valid))

        elapsedTime = (time.time() - start_time)/60
        print("--- %.2f minutes ---" % elapsedTime)

        
        savename = 'model_dopp_ratio_%d_SNR_%d.h5' % (ratio, SNR)
        model.save(savename)

test

In [ ]:
del_f = [5,20,40,70]
SNRs = [-2,10,23]
for SNR in SNRs:
    for ratio in del_f:
        model_name = 'model_dopp_ratio_%d_SNR_%d.h5' % (ratio, SNR)
        folder_name = 'ratio_%d' % (ratio)

        #load trained model at set alpha and SNR
        %cd /content/drive/My Drive/NN\ Input/Doppler/AWGN/Train\ Data
        model = tf.keras.models.load_model(model_name)

        %cd /content/drive/My Drive/NN\ Input/Doppler/AWGN/Test\ Data
        
        %cd $folder_name 
        for snr in range(-2,24): #all snr values
            for i in range(1,11): #loop over 10 test sequences 
                matname = 'DVBSAPSK_pred_ratio_%d_%d_SNR_%d' % (ratio, i, snr)

                mat = spio.loadmat(matname, squeeze_me=True) #load data
                x_test = mat['train_data']

                predictions = model.predict(x_test) #make predictions

                #save predictions 
                matname = 'DVBSAPSK_out_ratio_%d_%d_SNR_%d_trnSNR_%d.mat' % (ratio, i, snr, SNR)
                spio.savemat(matname, {'pred': predictions})

        %cd ..